# Analyzing CIA Factbook Data Using SQL

This is an extension to the guided project on DataQuest:

"In this project, we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:
- population - The population as of 2015.
- population_growth - The annual population growth rate, as a percentage.
- area - The total land and water area.

In this guided project, we'll use SQL in Jupyter Notebook to explore and analyze data from this database."

In [1]:
# connect to database

%load_ext sql
%sql sqlite:///factbook.db

'Connected: @factbook.db'

In [2]:
%%sql

SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT * FROM facts LIMIT 5; -- # let's see what the data looks like in the "facts" table

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [4]:
%%sql

-- # let's see what the min/max population and population growth are in the table

SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


- There's a country with a population of 0
- There's a country with a population of 7.2 billion people

In [5]:
%%sql

SELECT *
    FROM Facts
    WHERE population == (SELECT MIN(population) FROM Facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [6]:
%%sql

SELECT *
    FROM Facts
    WHERE population == (SELECT Max(population) FROM Facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


- The country with a population of 0 is Antarctica,
- The country with a population of 7.2bln people is the World. 

We need to recalculate summary statistics while excluding these two rows.

In [8]:
%%sql

SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts
    WHERE population != (SELECT MAX(population) FROM Facts) 
    AND population != (SELECT MIN(population)FROM Facts);

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
48,1367485388,0.0,4.02


Now let's find countries that are densely populated. We'll identify countries that have:
- Above average values for population.
- Below average values for area.

In [10]:
%%sql
SELECT AVG(population), AVG(area)
    FROM facts
    WHERE population != (SELECT MAX(population) FROM Facts) 
    AND population != (SELECT MIN(population) FROM Facts);

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
32377011.0125,582949.8523206752


In [12]:
%%sql

SELECT name
    FROM facts
    WHERE population > (SELECT AVG(population) FROM facts WHERE population != (SELECT MAX(population) FROM Facts) AND population != (SELECT MIN(population) FROM Facts)) 
    AND area < (SELECT AVG(area) FROM facts WHERE population != (SELECT MAX(population) FROM Facts) AND population != (SELECT MIN(population) FROM Facts));

 * sqlite:///factbook.db
Done.


name
Bangladesh
Germany
Iraq
Italy
Japan
Kenya
"Korea, South"
Morocco
Philippines
Poland


These are the countries with population above average and area below average.

Now, let's try answering the following questions:

### What country has the most people? What country has the highest growth rate?

In [16]:
%%sql

SELECT name
    FROM facts
    WHERE population < (SELECT MAX(population) FROM facts)
    ORDER BY population DESC
    LIMIT 1;

 * sqlite:///factbook.db
Done.


name
China


In [14]:
%%sql

SELECT name
    FROM facts
    WHERE population_growth = (SELECT MAX(population_growth) FROM facts);

 * sqlite:///factbook.db
Done.


name
South Sudan


### Which countries have the highest ratios of water to land? Which countries have more water than land?

In [18]:
%%sql

SELECT name, ROUND(CAST(area_water as Float)/CAST(area_land as Float),2) AS water_to_land
    FROM facts
    ORDER BY 2 DESC
    LIMIT 1;

 * sqlite:///factbook.db
Done.


name,water_to_land
British Indian Ocean Territory,905.67


In [19]:
%%sql

SELECT name
    FROM facts
    WHERE area_water>area_land;

 * sqlite:///factbook.db
Done.


name
British Indian Ocean Territory
Virgin Islands


### Which countries will add the most people to their population next year?

In [24]:
%%sql

SELECT name, ROUND(population*population_growth,0) AS added_people
    FROM facts
    WHERE population != (SELECT MAX(population) FROM facts)
    AND population != (SELECT MIN(population) FROM facts)
    ORDER BY 2 DESC
    LIMIT 10;    

 * sqlite:///factbook.db
Done.


name,added_people
India,1527068612.0
China,615368425.0
Nigeria,444827037.0
Pakistan,290665337.0
Ethiopia,287456217.0
Bangladesh,270332392.0
United States,250667714.0
Indonesia,235514180.0
"Congo, Democratic Republic of the",194469083.0
Philippines,162607385.0


### Which countries have a higher death rate than birth rate?

In [25]:
%%sql

SELECT name
    FROM facts
    WHERE death_rate>birth_rate;

 * sqlite:///factbook.db
Done.


name
Austria
Belarus
Bosnia and Herzegovina
Bulgaria
Croatia
Czech Republic
Estonia
Germany
Greece
Hungary


### What countries have the highest population/area ratio?

In [26]:
%%sql 

SELECT name, ROUND(population/area,1) AS pop_to_area
    FROM facts
    WHERE population != (SELECT MAX(population) FROM facts)
    AND population != (SELECT MIN(population) FROM facts)
    ORDER BY 2 DESC
    LIMIT 10;

 * sqlite:///factbook.db
Done.


name,pop_to_area
Macau,21168.0
Monaco,15267.0
Singapore,8141.0
Hong Kong,6445.0
Gaza Strip,5191.0
Gibraltar,4876.0
Bahrain,1771.0
Maldives,1319.0
Malta,1310.0
Bermuda,1299.0
